In [1]:
#Import Dependencies
import pandas as pd
import os
import csv

In [2]:
#Set variable for the input file
file_to_load="Resources/purchase_data.csv"
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#Count the total first, then use the drop_duplicate to verify if there is any duplicate values
purchase_data.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [4]:
#Find the total number of Players, which is the count of SN column, but there maybe some duplicate values
#so I will try to verify.
cleaned_data_df=purchase_data.drop_duplicates("SN")
cleaned_data_df.count()

Purchase ID    576
SN             576
Age            576
Gender         576
Item ID        576
Item Name      576
Price          576
dtype: int64

In [5]:
#Found that there is one SN id that purchased more than one, so the total number players would be the lenght of the SN column
Total_Players_df=pd.DataFrame([{"Total Players":len(cleaned_data_df['SN'].value_counts())}])
Total_Players_df

,Total Players
0,576


In [6]:
#Purchasing Analysis
Purchasing_Analysis_df=pd.DataFrame([
    {"Number of Unique Items":len(purchase_data['Item Name'].value_counts()),
    "Average Price":round(purchase_data['Price'].mean(),2),
    "Number of Purchases":len(purchase_data['Purchase ID']),
    "Total Revenue":purchase_data['Price'].sum()}])
Purchasing_Analysis_df["Average Price"]=Purchasing_Analysis_df["Average Price"].astype(float).map("${:,.2f}".format)
Purchasing_Analysis_df["Total Revenue"]=Purchasing_Analysis_df["Total Revenue"].astype(float).map("${:,.2f}".format)
Purchasing_Analysis_df.head()


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [7]:
#Gender Demographics
#Since there is some players represented more than one, so the gender would have the duplicate values
#so I would count the gender with the cleaned_data_df
gender_df = pd.DataFrame(cleaned_data_df["Gender"].value_counts())

gender_df.set_index("Gender")

gender_df.columns = ["Total Count"]

gender_df["Percentage of Players"]=round((gender_df['Total Count']/576)*100,2)
gender_df["Percentage of Players"]=gender_df["Percentage of Players"].astype(float).map("{0:.2f}%".format)
gender_df.head()




,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [8]:
#Purchasing Analysis based on Gender






